In [8]:
import json
import os
from pathlib import Path

import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm

In [9]:
api_key = "c07526f63bee4d248b5e3ce31f9d3435"
api_path = f"https://api.opencagedata.com/geocode/v1/json"

In [10]:
def get_proper_api_path(place, api_key=api_key, api_path=api_path):
    return f"{api_path}?q={place}&key={api_key}"

In [11]:
def fetch_place(place, **params):
    path = get_proper_api_path(place, **params)
    res = requests.get(path)
    
    if res.status_code != 200:
        print(f"Cannot fetch {place}")
        raise ValueError

    return json.loads(res.content.decode("utf8"))

In [12]:
bangla_path = 'https://en.wikipedia.org/wiki/List_of_airports_in_Bangladesh'
res = requests.get(bangla_path)
html = res.content.decode("utf8")
soup = BeautifulSoup(html, "html")
airports = []
rows = soup.find_all("table")
idxs = (1,0,3,6)

In [13]:
for i in range(3,10):
    tds = rows[i].find_all("td")
    length = int(len(tds)/7)
    if length == 8:
        length = 2
    for j in range(length):
        k0 = j*7;
        if (tds[2+k0].contents[0].rstrip() != '—' and tds[3+k0].contents[0].rstrip() != '—'):
            if (tds[4+k0].contents[0].rstrip() == 'International' or tds[4+k0].contents[0].rstrip() == 'Domestic'):
                href = 'https://en.wikipedia.org/'+tds[1+k0].contents[0].attrs['href']
                res1 = requests.get(href)
                html1 = res1.content.decode('utf8')
                soup1 = BeautifulSoup(html1,'html')
                rows1 = soup1.find("table").find_all("tr")[1:]
                passengers = rows1[len(rows1)-2].contents[1].text
                airport = {
                    "name": tds[idxs[0]+k0].a.string,
                    "city": tds[idxs[1]+k0].a.string,
                    "country": tds[idxs[2]+k0].contents[0].rstrip(),
                    "passengers": int(str(passengers).replace(",", ""))
                }
                query = f"{airport['city']},{airport['country']}"
                res_json = fetch_place(query)["results"]
                coords = res_json[0]["geometry"]
                airport["lng"] = coords["lng"]
                airport["lat"] = coords["lat"]
                airports.append(airport)

In [14]:
bangla_df = pd.DataFrame(airports)
data_path = Path(os.getcwd()).parent/"data"
bangla_df.to_csv(data_path/"bangla_airports.csv")